In [ ]:
import torch
import numpy as np
import pandas as pd
_ = torch.manual_seed(0)

In [ ]:
d, k = 10, 10

# This way we can generate a rank-deficient matrix
W_rank = 2
W = torch.randn(d, W_rank) @ torch.randn(W_rank, k)
pd.DataFrame(W)

In [ ]:
# Evaluate the rank of the matrix W

W_rank = np.linalg.matrix_rank(W)
print(f"Rank of matrix W: {W_rank}")

#### Calculate the SVD of the W matrix
Perform SVD on ${W = U*V*S^T}$

In [ ]:
U, S, V = torch.svd(W)

# For rank-r factorization, keep only the first r singular values (and corresponding columns of U and V)
U_r = U[: , :W_rank]
S_r = torch.diag(S[:W_rank])
V_r = V[:, :W_rank].t() # Transpose


# Compute C = U_r * S_r and R = V_r
B = U_r @ S_r
A = V_r
print(f"Shape of B: {B.shape}\nShape of A: {A.shape}")


#### Given the same input, check the output using the original W matrix and the matrices resulting from the decomposition.

In [ ]:
# Generate random bias and input
bias = torch.randn(d)
x = torch.randn(k)

# Compute y = Wx + b
y = W @ x + bias
y_prime = (B @ A) @ x + bias

print(f"Original y usgin W:\n{y}\n")
print(f"y usgin BA:\n{y_prime}")

In [ ]:
print(f"Total parameters of W: {W.nelement()}")
print(f"Total parameters of b and A: {B.nelement() + A.nelement()}")

In [ ]:
import time

import evaluate
import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
from peft import LoraConfig, TaskType, get_peft_model
from transformers import (AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments)

device = "mps"
weights_path: str = "./weights"
tokeniz_path: str = "./tokenizer"
datasets_path: str = "./dataset"
huggingface_dataset_name: str = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name, cache_dir=datasets_path)

print(dataset)


model_name = "google/flan-t5-base"
original_model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, torch_dtype=torch.float32, cache_dir=weights_path
)  # bfloat16
original_model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=tokeniz_path)


def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"


print(print_number_of_trainable_model_parameters(original_model))


index = 200

dialogue = dataset["test"][index]["dialogue"]
summary = dataset["test"][index]["summary"]

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors="pt").to(device)
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True,
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

In [3]:
import numpy as np
import pandas as pd


mbpp = 'https://raw.githubusercontent.com/google-research/google-research/master/mbpp/sanitized-mbpp.json'


df = pd.read_json(mbpp)


In [11]:
df.head(10)

,source_file,task_id,prompt,code,test_imports,test_list
0,Benchmark Questions Verification V2.ipynb,2,Write a function to find the shared elements f...,"def similar_elements(test_tup1, test_tup2):\n ...",[],"[assert set(similar_elements((3, 4, 5, 6),(5, ..."
1,Benchmark Questions Verification V2.ipynb,3,Write a python function to identify non-prime ...,import math\ndef is_not_prime(n):\n result ...,[],"[assert is_not_prime(2) == False, assert is_no..."
2,Benchmark Questions Verification V2.ipynb,4,Write a function to find the n largest integer...,import heapq as hq\ndef heap_queue_largest(num...,[],"[assert heap_queue_largest( [25, 35, 22, 85, 1..."
3,Benchmark Questions Verification V2.ipynb,6,Write a python function to check whether the t...,def is_Power_Of_Two (x): \n return x and (n...,[],"[assert differ_At_One_Bit_Pos(13,9) == True, a..."
4,Benchmark Questions Verification V2.ipynb,7,Write a function to find all words which are a...,import re\ndef find_char_long(text):\n return...,[],[assert set(find_char_long('Please move back t...
5,Benchmark Questions Verification V2.ipynb,8,Write a function to find squares of individual...,def square_nums(nums):\n square_nums = list(ma...,[],"[assert square_nums([1, 2, 3, 4, 5, 6, 7, 8, 9..."
6,Benchmark Questions Verification V2.ipynb,9,Write a python function to find the minimum nu...,def find_Rotations(str): \n tmp = str + str...,[],"[assert find_Rotations(""aaaa"") == 1, assert fi..."
7,Benchmark Questions Verification V2.ipynb,11,Write a python function to remove first and la...,"def remove_Occ(s,ch): \n for i in range(len...",[],"[assert remove_Occ(""hello"",""l"") == ""heo"", asse..."
8,Benchmark Questions Verification V2.ipynb,12,Write a function to sort a given matrix in asc...,"def sort_matrix(M):\n result = sorted(M, ke...",[],"[assert sort_matrix([[1, 2, 3], [2, 4, 5], [1,..."
9,Benchmark Questions Verification V2.ipynb,14,Write a python function to find the volume of ...,"def find_Volume(l,b,h) : \n return ((l * b ...",[],"[assert find_Volume(10,8,6) == 240, assert fin..."


In [7]:
def format_task(instruction):
        PROMPT_DICT = {
            "prompt_no_input": (
                "Below is an instruction that describes a task. "
                "Write a response that appropriately completes the request.\n\n"
                f"### Instruction:\n{instruction}\n\n### Response:"
            ),
        }
        return PROMPT_DICT['prompt_no_input']

format_task("WILTON")

'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nWILTON\n\n### Response:'

In [10]:
from datasets import load_dataset

ds = load_dataset('code_x_glue_ct_code_to_text', 'python')
ds



DatasetDict({
    train: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 251820
    })
    validation: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 13914
    })
    test: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 14918
    })
})